## **Install Library**

In [ ]:
%pip install selenium
%pip install pandas
%pip install requests
%pip install os

## **Scrapping Beberapa Provinsi dan Beberapa Range Page**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import urllib3

# Nonaktifkan peringatan InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Buat range provinsi dan halaman
provinsi_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 99]  # Daftar provinsi
halaman_list = list(range(1, 3))  # Pastikan nilai terbesar adalah 3940

# Membuat direktori jika belum ada
directory = "Hasil_Scrap"
if not os.path.exists(directory):
    os.makedirs(directory)

# Fungsi untuk menyimpan data ke CSV
def save_to_csv(provinsi_data, provinsi, halaman):
    df_provinsi = pd.DataFrame(provinsi_data)
    final_file_path = os.path.join(directory, f'provinsi_{provinsi}_hal_{halaman}.csv')
    df_provinsi.to_csv(final_file_path, index=False)
    print(f"Data dari halaman 1 hingga {halaman} untuk provinsi {provinsi} telah disimpan ke {final_file_path}")

# Loop untuk setiap provinsi
for provinsi in provinsi_list:
    provinsi_data = {'No': [],
                     'Perusahaan': [],
                     'Alamat': [],
                     'Nomor Telp': [],
                     'KBLI': []
                    }
    # Loop untuk setiap halaman
    for i, halaman in enumerate(halaman_list, start=1):
        url = f"https://kemenperin.go.id/direktori-perusahaan?what=&prov={provinsi}&hal={halaman}" ##url dari kemenperin
        try:
            response = requests.get(url, verify=False)
            response.raise_for_status()
        except requests.HTTPError as e:
            print(f"HTTP error occurred: {e}")
            continue
        except requests.RequestException as e:
            print(f"An error occurred: {e}")
            continue
        
        soup = BeautifulSoup(response.text, 'html.parser')
        div_tb = soup.find('div', {'class':'col-md-12 col-lg-12 col-xs-12 col-sm-12'})

        if div_tb:
            table = soup.find('table', {'id': 'newspaper-a'})
            if table:
                for row in table.find_all('tr'):
                    cells = row.find_all('td')
                    if len(cells) >= 3:
                        perusahaan = cells[1].find('b').text.strip()
                        alamat_tel = cells[1].find_all(text=True, recursive=False)
                        alamat = alamat_tel[0].strip() if len(alamat_tel) > 0 else ''
                        nomor_telp = alamat_tel[1].strip() if len(alamat_tel) > 1 else ''
                        kbli = cells[2].text.strip()

                        provinsi_data['No'].append(cells[0].text.strip())
                        provinsi_data['Perusahaan'].append(perusahaan)
                        provinsi_data['Alamat'].append(alamat)
                        provinsi_data['Nomor Telp'].append(nomor_telp)
                        provinsi_data['KBLI'].append(kbli)
                        print(halaman)
            else:
                print(f"Tidak ada tabel pada halaman {url}")
        else:
            print(f"Tidak ada tag Div pada halaman {url}")

        # Simpan data setiap 100 halaman
        if i % 100 == 0:
            save_to_csv(provinsi_data, provinsi, halaman)

    # Simpan data terakhir jika belum disimpan
    if provinsi_data['No']:
        save_to_csv(provinsi_data, provinsi, halaman)

print("Pengumpulan data selesai.")

## **Melakukan Combine Semua CSV menjadi 1 CSV**

In [ ]:
import os
import pandas as pd
import re

#Merujuk pada directory tempat save semua hasil scrapping per provinsi
directory = 'Hasil_Scrap' 

#ekstrak angka dari nama file
def extract_numbers(filename):
    match = re.search(r'provinsi_(\d+)_hal_(\d+)\.csv', filename)
    if match:
        provinsi = int(match.group(1))
        halaman = int(match.group(2))
        return (provinsi, halaman)
    return (float('inf'), float('inf'))  # Jika tidak cocok, letakkan di akhir

# list untuk menyimpan data frame dan nama file
dataframes = []

# Loop melalui semua file di dalam direktori
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        dataframes.append((file_path, extract_numbers(filename)))

# Urutkan daftar berdasarkan nomor provinsi dan halaman
dataframes.sort(key=lambda x: x[1])

# Buat daftar untuk menyimpan data frame yang diurutkan
sorted_dataframes = [pd.read_csv(file[0]) for file in dataframes]

# Gabungkan semua data frame menjadi satu
combined_df = pd.concat(sorted_dataframes, ignore_index=True)

# Simpan data frame gabungan ke dalam satu file CSV
output_file = os.path.join(directory, 'combined_data.csv')
combined_df.to_csv(output_file, index=False)

print(f"Semua file CSV telah digabungkan dan disimpan dalam {output_file}")
